In [0]:
import os
import sys

import networkx as nx
from GraphReasoning import (
    AutoModel,
    AutoTokenizer,
    find_path_and_reason,
    generate_node_embeddings,
    load_embeddings,
    make_graph_from_text,
    save_embeddings,
)
from IPython.display import Markdown, display
from openai import OpenAI

from core.constants import LLMConstants

sys.path.append("..")

In [ ]:
client = OpenAI(api_key=LLMConstants.OPENAI_API_KEY)

### Load graph and embeddings 

In [0]:
def complete_message_with_4o(system_prompt, prompt, temperature=0.333, max_tokens=4096):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            { "role": "system", "content": system_prompt },
            { "role": "user", "content": [ {"type": "text", "text": prompt} ] },
        ],
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message.content

In [0]:
#Hugging Face repo
repository_id = "lamm-mit/GraphReasoning"

data_dir="./GRAPHDATA"
text_input_dir="./TEXT_INPUT/"

data_dir_output="./GRAPHDATA_OUTPUT/"

tokenizer_model="BAAI/bge-large-en-v1.5"

embedding_tokenizer = AutoTokenizer.from_pretrained(tokenizer_model, )
embedding_model = AutoModel.from_pretrained(tokenizer_model, )

graph_root = "knowledgebasecomplete"
graph_text_file_name = "knowledgebasecondensed_2024-11-30.txt"
graph_name=f'{data_dir}/{graph_root}_graphML.graphml'

if os.path.exists(graph_name):
    G = nx.read_graphml(graph_name)

else:
    with open(os.path.join(text_input_dir, graph_text_file_name), "r") as f:
        text = f.read()
        graph_HTML, graph_GraphML, G, net, _ = make_graph_from_text(
            txt=text,
            graph_root=graph_root,
            generate=complete_message_with_4o,
            data_dir=data_dir,
            chunk_size=2500,
        )

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

In [0]:
embedding_file = f"{graph_root}_embeddings_ge-large-en-v1.5.pkl"
embedding_path = f"{data_dir}/{embedding_file}"

if os.path.exists(embedding_path):
    node_embeddings = load_embeddings(embedding_path)

else:
    node_embeddings = generate_node_embeddings(G, embedding_tokenizer, embedding_model, )
    save_embeddings(node_embeddings, embedding_path)

In [0]:
response, (best_node_1, best_similarity_1, best_node_2, best_similarity_2), path, path_graph, shortest_path_length, fname, graph_GraphML = find_path_and_reason(
    G,
    node_embeddings,
    embedding_tokenizer,
    embedding_model,
    complete_message_with_4o,
    data_dir=data_dir_output,
    verbatim=True,
    include_keywords_as_nodes=True,  # Include keywords in the graph analysis
    keyword_1="parkinson's disease",
    keyword_2="treatment groups",
    N_limit=9999,  # The limit for keywords, triplets, etc.
    instruction='Develop a new research idea on a new drug that can help mitigate Parkinson desease.',
    keywords_separator=', ',
    graph_analysis_type='nodes and relations',
    temperature=0.3,
    inst_prepend='### ',  # Instruction prepend text
    prepend='''You are given a set of information from a graph that describes the relationship 
               between materials, structure, properties, and properties. You analyze these logically 
               through reasoning.\n\n''',  # Prepend text for analysis
    visualize_paths_as_graph=True,  # Whether to visualize paths as a graph
    display_graph=True,  # Whether to display the graph
)
display(Markdown(response))

0nth best fitting node for 'parkinson's disease': 'parkinson's disease' with similarity: 1
0nth best fitting node for 'treatment groups': 'treatment groups' with similarity: 1
./GRAPHDATA_OUTPUT//shortest_path_2hops_parkinson's disease_treatment groups.html
HTML visualization: ./GRAPHDATA_OUTPUT//shortest_path_2hops_parkinson's disease_treatment groups.html
GraphML file: ./GRAPHDATA_OUTPUT//shortestpath_2hops_parkinson's disease_treatment groups.graphml
You are given a set of information from a graph that describes the relationship 
               between materials, structure, properties, and properties. You analyze these logically 
               through reasoning.

### Consider this list of nodes and relations in a knowledge graph:

Format: node_1, relationship, node_2

The data is:

mds-updrs iii score, Change in, placebo
phase 3 trial in the uk, is repurposed in, exenatide
drug development pipeline, robust for, parkinson's disease
prevalence, doubling, parkinson's disease
microbiom

**Response:** Based on the provided knowledge graph and the relationships between various nodes, a new research idea for a drug to mitigate Parkinson's disease could focus on targeting the neuroinflammatory processes and oxidative stress, which are both implicated in the progression of Parkinson's disease. 

### Research Idea: Development of a Dual-Action Drug Targeting Neuroinflammation and Oxidative Stress in Parkinson's Disease

#### Background:
1. **Neuroinflammation and Parkinson's Disease**: Neuroinflammatory processes accompany Parkinson's disease and contribute to its progression (neuroinflammatory process, accompanies, Parkinson's disease).
2. **Oxidative Stress and Parkinson's Disease**: Oxidative stress is involved in Parkinson's disease and is connected to neuroinflammation (Parkinson's disease, contributes to, oxidative stress; oxidative stress, connected to, neuroinflammation).
3. **Immunomodulatory Therapies**: Immunomodulatory therapies can halt degenerative diseases like Parkinson's disease by targeting neuroinflammation (immunomodulatory therapies, halt degenerative disease, Parkinson's disease; immunomodulatory therapies, target, neuroinflammation).

#### Hypothesis:
A drug that combines immunomodulatory properties with antioxidant capabilities could effectively reduce neuroinflammation and oxidative stress, thereby slowing the progression of Parkinson's disease.

#### Proposed Drug: **NeuroImmunoX**

**NeuroImmunoX** is a novel dual-action drug designed to mitigate Parkinson's disease by:
1. **Reducing Neuroinflammation**: Utilizing immunomodulatory agents to decrease the inflammatory response in the brain.
2. **Combating Oxidative Stress**: Incorporating potent antioxidants to neutralize free radicals and reduce oxidative damage.

#### Mechanism of Action:
1. **Immunomodulatory Component**: 
   - **Target**: Neuroinflammatory pathways, specifically targeting pro-inflammatory cytokines and microglial activation.
   - **Action**: Inhibits the release of pro-inflammatory cytokines (e.g., TNF-α, IL-1β) and reduces microglial activation, thereby decreasing neuroinflammation.

2. **Antioxidant Component**:
   - **Target**: Reactive oxygen species (ROS) and oxidative stress markers.
   - **Action**: Scavenges free radicals and upregulates endogenous antioxidant defenses (e.g., superoxide dismutase, glutathione), reducing oxidative stress.

#### Research Plan:
1. **Preclinical Studies**:
   - **In Vitro**: Test the efficacy of NeuroImmunoX on cultured neuronal cells and microglia to assess its ability to reduce neuroinflammation and oxidative stress.
   - **In Vivo**: Use animal models of Parkinson's disease (e.g., MPTP model) to evaluate the drug's neuroprotective effects, focusing on behavioral outcomes, neuroinflammatory markers, and oxidative stress levels.

2. **Clinical Trials**:
   - **Phase I**: Assess the safety, tolerability, and pharmacokinetics of NeuroImmunoX in healthy volunteers.
   - **Phase II**: Evaluate the efficacy and safety of NeuroImmunoX in patients with early-stage Parkinson's disease, with primary endpoints including reduction in neuroinflammatory markers and oxidative stress levels.
   - **Phase III**: Conduct a larger, randomized, placebo-controlled trial to confirm the efficacy of NeuroImmunoX in slowing disease progression and improving clinical outcomes in Parkinson's disease patients.

#### Expected Outcomes:
- **Reduction in Neuroinflammation**: Decreased levels of pro-inflammatory cytokines and reduced microglial activation in the brain.
- **Lower Oxidative Stress**: Reduced ROS levels and increased activity of endogenous antioxidant enzymes.
- **Improved Clinical Outcomes**: Slower progression of motor and non-motor symptoms of Parkinson's disease, improved quality of life, and reduced disability-adjusted life years (DALYs).

By targeting both neuroinflammation and oxidative stress, NeuroImmunoX has the potential to provide a comprehensive therapeutic approach to mitigate the progression of Parkinson's disease and improve patient outcomes.

Based on the provided knowledge graph and the relationships between various nodes, a new research idea for a drug to mitigate Parkinson's disease could focus on targeting the neuroinflammatory processes and oxidative stress, which are both implicated in the progression of Parkinson's disease. 

### Research Idea: Development of a Dual-Action Drug Targeting Neuroinflammation and Oxidative Stress in Parkinson's Disease

#### Background:
1. **Neuroinflammation and Parkinson's Disease**: Neuroinflammatory processes accompany Parkinson's disease and contribute to its progression (neuroinflammatory process, accompanies, Parkinson's disease).
2. **Oxidative Stress and Parkinson's Disease**: Oxidative stress is involved in Parkinson's disease and is connected to neuroinflammation (Parkinson's disease, contributes to, oxidative stress; oxidative stress, connected to, neuroinflammation).
3. **Immunomodulatory Therapies**: Immunomodulatory therapies can halt degenerative diseases like Parkinson's disease by targeting neuroinflammation (immunomodulatory therapies, halt degenerative disease, Parkinson's disease; immunomodulatory therapies, target, neuroinflammation).

#### Hypothesis:
A drug that combines immunomodulatory properties with antioxidant capabilities could effectively reduce neuroinflammation and oxidative stress, thereby slowing the progression of Parkinson's disease.

#### Proposed Drug: **NeuroImmunoX**

**NeuroImmunoX** is a novel dual-action drug designed to mitigate Parkinson's disease by:
1. **Reducing Neuroinflammation**: Utilizing immunomodulatory agents to decrease the inflammatory response in the brain.
2. **Combating Oxidative Stress**: Incorporating potent antioxidants to neutralize free radicals and reduce oxidative damage.

#### Mechanism of Action:
1. **Immunomodulatory Component**: 
   - **Target**: Neuroinflammatory pathways, specifically targeting pro-inflammatory cytokines and microglial activation.
   - **Action**: Inhibits the release of pro-inflammatory cytokines (e.g., TNF-α, IL-1β) and reduces microglial activation, thereby decreasing neuroinflammation.

2. **Antioxidant Component**:
   - **Target**: Reactive oxygen species (ROS) and oxidative stress markers.
   - **Action**: Scavenges free radicals and upregulates endogenous antioxidant defenses (e.g., superoxide dismutase, glutathione), reducing oxidative stress.

#### Research Plan:
1. **Preclinical Studies**:
   - **In Vitro**: Test the efficacy of NeuroImmunoX on cultured neuronal cells and microglia to assess its ability to reduce neuroinflammation and oxidative stress.
   - **In Vivo**: Use animal models of Parkinson's disease (e.g., MPTP model) to evaluate the drug's neuroprotective effects, focusing on behavioral outcomes, neuroinflammatory markers, and oxidative stress levels.

2. **Clinical Trials**:
   - **Phase I**: Assess the safety, tolerability, and pharmacokinetics of NeuroImmunoX in healthy volunteers.
   - **Phase II**: Evaluate the efficacy and safety of NeuroImmunoX in patients with early-stage Parkinson's disease, with primary endpoints including reduction in neuroinflammatory markers and oxidative stress levels.
   - **Phase III**: Conduct a larger, randomized, placebo-controlled trial to confirm the efficacy of NeuroImmunoX in slowing disease progression and improving clinical outcomes in Parkinson's disease patients.

#### Expected Outcomes:
- **Reduction in Neuroinflammation**: Decreased levels of pro-inflammatory cytokines and reduced microglial activation in the brain.
- **Lower Oxidative Stress**: Reduced ROS levels and increased activity of endogenous antioxidant enzymes.
- **Improved Clinical Outcomes**: Slower progression of motor and non-motor symptoms of Parkinson's disease, improved quality of life, and reduced disability-adjusted life years (DALYs).

By targeting both neuroinflammation and oxidative stress, NeuroImmunoX has the potential to provide a comprehensive therapeutic approach to mitigate the progression of Parkinson's disease and improve patient outcomes.